In [2]:
# load data from 
from advmdata.nortek import AquadoppADVMData

data_directory = r'GCMRC data\2018.02.22'
dataset_name = 'AQD_225_2MHz_110217_121517.prf'
aqd_advm_data = AquadoppADVMData.read_aquadopp_data(data_directory, dataset_name)

from said.acoustic import RawBackscatterData

abs_backscatter_data = RawBackscatterData.from_advm_data(aqd_advm_data)
abs_data_df = abs_backscatter_data.get_data()

C:\Users\domanski\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# load the MATLAB generated data
import pandas as pd
aqd_amp_vel_df = pd.read_table(r'GCMRC data\2018.02.22\225MileGC_2MHz_AQD_amp_vel_05-Mar-2018.txt', index_col=False)

old_date_time_columns = [' Year ', ' Month ', ' Day ', ' Hour ', ' Min ', ' Sec ']
date_time_columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']
date_time_columns_map = dict(zip(old_date_time_columns, date_time_columns))

amp_columns_map = dict([(' Counts{:1},{:1} '.format(beam, cell), 'Cell{:02}Amp{:1}'.format(cell, beam)) for cell in range(1, 51) for beam in range(1,3)])

rename_columns_map = {' Temperature ': 'Temp'}
rename_columns_map.update(date_time_columns_map)
rename_columns_map.update(amp_columns_map)

aqd_amp_vel_df.rename(columns=rename_columns_map, inplace=True)

datetime_index = pd.to_datetime(aqd_amp_vel_df[date_time_columns])
aqd_amp_vel_df.set_index(datetime_index, inplace=True)

aqd_amp_vel_df = aqd_amp_vel_df.filter(regex=AquadoppADVMData._advm_columns_regex)

aqd_amp_vel_df = aqd_amp_vel_df.iloc[(abs_data_df.index[0] <= aqd_amp_vel_df.index) & (aqd_amp_vel_df.index <= abs_data_df.index[-1])]

In [4]:
import numpy as np
np.all(aqd_amp_vel_df.index.intersection(abs_data_df.index) == aqd_amp_vel_df.index.union(abs_data_df.index))

True

In [5]:
columns_intersection = aqd_amp_vel_df.columns.intersection(abs_data_df.columns)
columns_intersection

Index(['Temp', 'Cell01Amp1', 'Cell02Amp1', 'Cell03Amp1', 'Cell04Amp1',
       'Cell05Amp1', 'Cell06Amp1', 'Cell07Amp1', 'Cell08Amp1', 'Cell09Amp1',
       ...
       'Cell41Amp2', 'Cell42Amp2', 'Cell43Amp2', 'Cell44Amp2', 'Cell45Amp2',
       'Cell46Amp2', 'Cell47Amp2', 'Cell48Amp2', 'Cell49Amp2', 'Cell50Amp2'],
      dtype='object', length=101)

In [6]:
((aqd_amp_vel_df[columns_intersection] - abs_data_df[columns_intersection]).max()).max()

0.0